In [40]:
import tensorflow as tf
import tensorflow_datasets as tfds

데이터 셋 로드 (TFDS에서 가져온 EMNIST)

In [41]:
(etfds_train, etfds_test), etfds_info = tfds.load("emnist", split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
print(etfds_info)

def normalize_img_ascii_lable(image, label):
    ascii_Lb = label
    if ascii_Lb >= 10:
        ascii_Lb += 8
    if ascii_Lb >= 43:
        ascii_Lb += 6
    ascii_Lb += 15
    return tf.cast(image, tf.float32) / 255., ascii_Lb

etfds_train = etfds_train.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_train = etfds_train.cache()
etfds_train = etfds_train.shuffle(etfds_info.splits['train'].num_examples)
etfds_train = etfds_train.batch(128)
etfds_train = etfds_train.prefetch(tf.data.experimental.AUTOTUNE)

etfds_test = etfds_test.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_test = etfds_test.batch(128)
etfds_test = etfds_test.cache()
etfds_test = etfds_test.prefetch(tf.data.experimental.AUTOTUNE)

tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/byclass/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST ByClass
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_path='/Users/varzero/tensorflow_datasets/emnist/byclass/3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=349.16 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=62),
    }),
    supe

직접 만든 데이터 셋 로드

In [42]:
datasetImgs = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255., validation_split=0.1)
trainFG = datasetImgs.flow_from_directory(directory="../dataset/", target_size=(28,28), batch_size=10, class_mode='binary', subset='training', shuffle=True, color_mode='grayscale')
testFG = datasetImgs.flow_from_directory(directory="../dataset/", target_size=(28,28), batch_size=10, class_mode='binary', subset='validation', shuffle=True, color_mode='grayscale')

print(trainFG)

Found 1425 images belonging to 95 classes.
Found 95 images belonging to 95 classes.


모델 세팅

In [43]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(95, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 95)                48735     
                                                                 
Total params: 450,655
Trainable params: 450,655
Non-trainable params: 0
_________________________________________________________________


훈련시키기

In [44]:
model.fit(trainFG, epochs=5)

model.fit(etfds_train, epochs=5)


Epoch 1/5


2023-07-18 00:35:46.364680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


143/143 [==============================] - 3s 11ms/step - loss: 4.2566 - accuracy: 0.0919
Epoch 2/5
143/143 [==============================] - 1s 7ms/step - loss: 2.5973 - accuracy: 0.5011
Epoch 3/5
143/143 [==============================] - 1s 8ms/step - loss: 1.4829 - accuracy: 0.7081
Epoch 4/5
143/143 [==============================] - 1s 8ms/step - loss: 0.9086 - accuracy: 0.8407
Epoch 5/5
143/143 [==============================] - 1s 8ms/step - loss: 0.5820 - accuracy: 0.9151
Epoch 1/5


2023-07-18 00:35:54.315022: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [4]
	 [[{{node Placeholder/_0}}]]
2023-07-18 00:35:54.316650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [4]
	 [[{{node Placeholder/_4}}]]


5453/5453 [==============================] - 90s 9ms/step - loss: 0.7772 - accuracy: 0.7699
Epoch 2/5
5453/5453 [==============================] - 53s 10ms/step - loss: 0.5333 - accuracy: 0.8209
Epoch 3/5
5453/5453 [==============================] - 57s 10ms/step - loss: 0.4953 - accuracy: 0.8307
Epoch 4/5
5453/5453 [==============================] - 47s 8ms/step - loss: 0.4758 - accuracy: 0.8353
Epoch 5/5
5453/5453 [==============================] - 48s 9ms/step - loss: 0.4637 - accuracy: 0.8383


모델 저장하기

In [45]:
model.save('spemnist.h5')

확인하기

In [46]:
model.evaluate(etfds_test, verbose=2)
model.evaluate(testFG, verbose=2)

2023-07-18 00:41:48.496352: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-07-18 00:41:48.497303: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


909/909 - 11s - loss: 0.4468 - accuracy: 0.8446 - 11s/epoch - 13ms/step


2023-07-18 00:42:00.065410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 - 0s - loss: 10.1966 - accuracy: 0.0105 - 324ms/epoch - 32ms/step


[10.196556091308594, 0.010526316240429878]